In [ ]:
import pandas as pd
import numpy as np
import math
import string
from collections import Counter
from google.colab import files
from numpy import mean
from numpy import std
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import NearMiss
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
uploaded = files.upload()

Saving input.csv to input (1).csv


In [ ]:
# read input file. Input.csv has 6111 rows, Input1.csv has 32418 rows
df = pd.read_csv("input.csv")
# df = pd.read_csv("input1.csv")
df

,Plaintext,Ciphertext,Cipher Class
0,readmedos,SFBENFEPT,0
1,interrupt,JOUFSSVQU,0
2,reporters,SFQPSUFST,0
3,australia,BVTUSBMJB,0
4,cancernet,DBODFSOFU,0
...,...,...,...
6106,oldtimers,SIFBBURXW,2
6107,drawbacks,HOCEUIPQW,2
6108,cleveland,GIGDXTNTH,2
6109,viewpoint,ZFGEIWVTX,2


### Features engineering

In [ ]:
# I used "ACA Reference Statistcs" to compute features from ciphertext
# https://bionsgadgets.appspot.com/gadget_forms/acarefstats.html

# Index of coincidence (IC)
def get_letter_counts(text):
    text_upper = text.upper()
    letter_counts = {}
    for index, letter in enumerate(string.ascii_uppercase):
        letter_counts[letter] = text_upper.count(letter)
    return letter_counts
def get_letter_frequencies(text):
    letter_counts = get_letter_counts(text)
    frequencies = {letter: count/len(text) for letter, count in letter_counts.items()}
    return frequencies
def get_ic(letter_counts):
    numerator = sum([letter_counts[l]*(letter_counts[l]-1) for l in string.ascii_uppercase])
    text_size = sum(occurrences for occurrences in letter_counts.values())
    denominator = text_size*(text_size-1)
    return numerator/denominator

# Number of unique characters (NUC)
def get_nuc(str):
    # Stores all frequencies
    freq = Counter(str)
    # Return the size of the freq dictionary
    return len(freq)

# I just only implemented 2 simple features which are not really significant features, 
# I think MIC and DIC mentioned in the link above are significant features to detect those ciphers because they contain
# the frequence of letters in cipher text. I am not understand the algorithm yet, so I cannot implement them.
# the number of features will be reduced if we have MIC and DIC features, instead of using 26 alphabec letter as features

In [ ]:
#TEST

tmp1 = get_letter_counts("SFBENFEPT")
tmp2 = get_letter_counts("JOUFSSVQU")
tmp1

{'A': 0,
 'B': 1,
 'C': 0,
 'D': 0,
 'E': 2,
 'F': 2,
 'G': 0,
 'H': 0,
 'I': 0,
 'J': 0,
 'K': 0,
 'L': 0,
 'M': 0,
 'N': 1,
 'O': 0,
 'P': 1,
 'Q': 0,
 'R': 0,
 'S': 1,
 'T': 1,
 'U': 0,
 'V': 0,
 'W': 0,
 'X': 0,
 'Y': 0,
 'Z': 0}

In [ ]:
#TEST
get_letter_frequencies("SFBENFEPT")

{'A': 0.0,
 'B': 0.1111111111111111,
 'C': 0.0,
 'D': 0.0,
 'E': 0.2222222222222222,
 'F': 0.2222222222222222,
 'G': 0.0,
 'H': 0.0,
 'I': 0.0,
 'J': 0.0,
 'K': 0.0,
 'L': 0.0,
 'M': 0.0,
 'N': 0.1111111111111111,
 'O': 0.0,
 'P': 0.1111111111111111,
 'Q': 0.0,
 'R': 0.0,
 'S': 0.1111111111111111,
 'T': 0.1111111111111111,
 'U': 0.0,
 'V': 0.0,
 'W': 0.0,
 'X': 0.0,
 'Y': 0.0,
 'Z': 0.0}

In [ ]:
#TEST
get_ic(get_letter_counts("SFBENFEPT"))

0.05555555555555555

In [ ]:
cipher_text = df["Ciphertext"].values
cipher_text

array(['SFBENFEPT', 'JOUFSSVQU', 'SFQPSUFST', ..., 'GIGDXTNTH',
       'ZFGEIWVTX', 'WRDAVZVHI'], dtype=object)

In [ ]:
# Features engineering functions
def feature_engineering(Dataframe, cipher_text):
  df = Dataframe
  ic = []
  nuc = []
  freq = []

  # extract features from each ciphertext datapoint
  for i in cipher_text:
    # *26.0 to normalize the values
    # ic.append(get_ic(get_letter_counts(i))*26.0)
    ic.append(get_ic(get_letter_counts(i)))
    nuc.append(get_nuc(i))
    freq.append(get_letter_counts(i))

  # add IC and NUC features into df
  df["ic"] = ic
  df["nuc"] = nuc

  # concat df (Plaintext,Ciphertext,Cipher Class) with alphabec features df1(A,B,..,Z)
  alpha_features_pd = defaultdict(list)
  for d in freq: # you can list as many input dicts as you want here
      for key, value in d.items():
          alpha_features_pd[key].append(value)
  df1 = pd.DataFrame(alpha_features_pd)
  df = pd.concat([df,df1], axis=1)
  return df


In [ ]:
df = feature_engineering(df,cipher_text)

In [ ]:
# Note: df should have the shape of (6111 rows × 31 columns), it may duplicate when you not runing from the begining when testing my code
df

,Plaintext,Ciphertext,Cipher Class,ic,nuc,A,B,C,D,E,...,Q,R,S,T,U,V,W,X,Y,Z
0,readmedos,SFBENFEPT,0,0.055556,7,0,1,0,0,2,...,0,0,1,1,0,0,0,0,0,0
1,interrupt,JOUFSSVQU,0,0.055556,7,0,0,0,0,0,...,1,0,2,0,2,1,0,0,0,0
2,reporters,SFQPSUFST,0,0.111111,6,0,0,0,0,0,...,1,0,3,1,1,0,0,0,0,0
3,australia,BVTUSBMJB,0,0.083333,7,0,3,0,0,0,...,0,0,1,1,1,1,0,0,0,0
4,cancernet,DBODFSOFU,0,0.083333,6,0,1,0,2,0,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6106,oldtimers,SIFBBURXW,2,0.027778,8,0,2,0,0,0,...,0,1,1,0,1,0,1,1,0,0
6107,drawbacks,HOCEUIPQW,2,0.000000,9,0,0,1,0,1,...,1,0,0,0,1,0,1,0,0,0
6108,cleveland,GIGDXTNTH,2,0.055556,7,0,0,0,1,0,...,0,0,0,2,0,0,0,1,0,0
6109,viewpoint,ZFGEIWVTX,2,0.000000,9,0,0,0,0,1,...,0,0,0,1,0,1,1,1,0,1


In [ ]:
# Note: I tested with both training dataset with ensemble methods. Model with Alphabec count + ic + nuc features 
# got the better acc 0.87, compare to 0.81 with only alphabec count feature. You can uncomment the 2nd training set to test with only alphabec feature
# It shows IC and NUC features helps to imrove the model accuracy.

X = df.loc[:,df.columns.difference(['Plaintext', 'Ciphertext','Cipher Class'])]
# X = df.loc[:,df.columns.difference(['Plaintext', 'Ciphertext','Cipher Class','ic','nuc'])]
y = df["Cipher Class"]
print(X.shape)
print(y.shape)
print(X.columns)

(6111, 28)
(6111,)
Index(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
       'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'ic',
       'nuc'],
      dtype='object')


In [ ]:
Counter(y)

Counter({0: 3395, 1: 1358, 2: 1358})

In [ ]:
X[0:5]

,A,B,C,D,E,F,G,H,I,J,...,S,T,U,V,W,X,Y,Z,ic,nuc
0,0,1,0,0,2,2,0,0,0,0,...,1,1,0,0,0,0,0,0,0.055556,7
1,0,0,0,0,0,1,0,0,0,1,...,2,0,2,1,0,0,0,0,0.055556,7
2,0,0,0,0,0,2,0,0,0,0,...,3,1,1,0,0,0,0,0,0.111111,6
3,0,3,0,0,0,0,0,0,0,1,...,1,1,1,1,0,0,0,0,0.083333,7
4,0,1,0,2,0,2,0,0,0,0,...,1,0,1,0,0,0,0,0,0.083333,6


In [ ]:
# without scaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify = y, shuffle=True)

In [ ]:
model_LR = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model_LR.fit(X_train, y_train)
y_hat_LR = model_LR.predict(X_test)
accuracy_score(y_test, y_hat_LR)


0.8544562551103843

#### I tested with Solver 'newton-cg' which uses Ridge reguralization. This solver gives a better performance, compared to other solvers in 
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
# with scaler, Z standardization
scaler= StandardScaler()
X =scaler.fit_transform(X,y) 
X[0:2]

array([[-0.4155145 ,  1.19445769, -0.50750884, -0.51932948,  2.77503265,
         2.37484476, -0.67163581, -0.63540834, -0.63399403, -0.50455622,
        -0.60439399, -0.52140572, -0.56956806,  1.3237784 , -0.57507785,
         1.07607006, -0.68980245, -0.62286832,  1.08832755,  0.84114034,
        -0.60274732, -0.77660389, -0.70332265, -0.5642174 , -0.54290819,
        -0.48315645,  0.31949845, -0.64070665],
       [-0.4155145 , -0.53606174, -0.50750884, -0.51932948, -0.61220884,
         0.86468073, -0.67163581, -0.63540834, -0.63399403,  1.14429035,
        -0.60439399, -0.52140572, -0.56956806, -0.5540326 ,  1.18735984,
        -0.62355882,  0.88017251, -0.62286832,  2.80288533, -0.69830519,
         2.81254141,  0.75430784, -0.70332265, -0.5642174 , -0.54290819,
        -0.48315645,  0.31949845, -0.64070665]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify = y, shuffle=True)

In [ ]:
model_LR = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model_LR.fit(X_train, y_train)
y_hat_LR = model_LR.predict(X_test)
print(accuracy_score(y_test, y_hat_LR))
print(confusion_matrix(y_test, y_hat_LR))

0.848732624693377
[[618   0  61]
 [  0 272   0]
 [124   0 148]]


In [ ]:
# Use decision tree
model_DTC = DecisionTreeClassifier().fit(X_train, y_train)
y_hat_DTC = model_DTC.predict(X_test)
print(accuracy_score(y_test,y_hat_DTC))
print(confusion_matrix(y_test, y_hat_DTC))

0.8405560098119379
[[587   8  84]
 [ 15 252   5]
 [ 80   3 189]]


### Handling imbalance dataset with sampling techniques

In [ ]:
# Up sampling with SMOTE
sm = SMOTE(k_neighbors=5)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)
print(Counter(y_train))
print(Counter(y_train_sm))

Counter({0: 2716, 1: 1086, 2: 1086})
Counter({0: 2716, 1: 2716, 2: 2716})


In [ ]:
model_LR_sm = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model_LR_sm.fit(X_train_sm, y_train_sm)
y_hat_LR_sm = model_LR_sm.predict(X_test)
print(accuracy_score(y_test, y_hat_LR_sm))
print(confusion_matrix(y_test, y_hat_LR_sm))

0.8291087489779232
[[538   0 141]
 [  0 272   0]
 [ 68   0 204]]


In [ ]:
model_DTC_sm = DecisionTreeClassifier().fit(X_train_sm, y_train_sm)
y_hat_DTC_sm = model_DTC_sm.predict(X_test)
print(accuracy_score(y_test, y_hat_DTC_sm))
print(confusion_matrix(y_test, y_hat_DTC_sm))

0.8470973017170891
[[593   3  83]
 [  0 264   8]
 [ 92   1 179]]


In [ ]:
# Down sampling with NearMiss
nm = NearMiss(n_neighbors=3)
X_train_nm, y_train_nm = nm.fit_resample(X_train, y_train)
print(Counter(y_train))
print(Counter(y_train_nm))

Counter({0: 2716, 1: 1086, 2: 1086})
Counter({0: 1086, 1: 1086, 2: 1086})


In [ ]:
model_LR_nm = LogisticRegression(multi_class='multinomial', solver='newton-cg')
model_LR_nm.fit(X_train_nm, y_train_nm)
y_hat_LR_nm = model_LR_sm.predict(X_test)
print(accuracy_score(y_test, y_hat_LR_nm))
print(confusion_matrix(y_test, y_hat_LR_nm))

0.8291087489779232
[[538   0 141]
 [  0 272   0]
 [ 68   0 204]]


In [ ]:
model_DTC_nm = DecisionTreeClassifier().fit(X_train_nm, y_train_nm)
y_hat_DTC_nm = model_DTC_nm.predict(X_test)
print(accuracy_score(y_test, y_hat_DTC_nm))
print(confusion_matrix(y_test, y_hat_DTC_nm))

0.7080948487326247
[[416  39 224]
 [ 13 246  13]
 [ 49  19 204]]


Down sampling with NearMiss is not help when dealing with small dataset. Class 0 is reduced to 1086, losing insights.

### Ensemble method
The result indicates that Decision Tree is not stable, when the data changes slightly causing the tree structure changes drastically. In addtion, the prediction of DT is not high due to the simplicity of the model. Now we use ensemble methods to tackle these issues.

In [ ]:
# Bagging without sampling 
model_BC = BaggingClassifier().fit(X_train, y_train)
y_hat_BC = model_BC.predict(X_test)
print(accuracy_score(y_test,y_hat_BC))
print(confusion_matrix(y_test,y_hat_BC))

0.8765331152902698
[[632   4  43]
 [  6 262   4]
 [ 92   2 178]]


In [ ]:
# Bagging with SMOTE sampling 
model_BC_sm = BaggingClassifier().fit(X_train_sm, y_train_sm)
y_hat_BC_sm = model_BC_sm.predict(X_test)
print(accuracy_score(y_test,y_hat_BC_sm))
print(confusion_matrix(y_test,y_hat_BC_sm))

0.8781684382665577
[[629   1  49]
 [  2 262   8]
 [ 89   0 183]]


In [ ]:
# Bagging with NEARMISS sampling
model_BC_nm = BaggingClassifier().fit(X_train_nm, y_train_nm)
y_hat_BC_nm = model_BC_nm.predict(X_test)
print(accuracy_score(y_test,y_hat_BC_nm))
print(confusion_matrix(y_test,y_hat_BC_nm))

0.7350776778413737
[[419  10 250]
 [  0 268   4]
 [ 59   1 212]]


We can see that with small dataset, down sampling with near miss does not help. However, when I tested with bigger dataset (you can use "input1.csv" to test), where class 1, and 2 are big enough, then the confusion matrix shows the model can handle better in each cipher class.

After testing the linear regression and decision tree, with different tuning hyperparamers such as max_dept, complexity alpha, solvers for linear model, up/down sampling, and ensembel method, Decision Tree ensemble methods seems give the best performance.

### Test with unseen dataset

In [ ]:
DrShah = files.upload()

Saving test dataset sample.csv to test dataset sample.csv


In [ ]:
df1 = pd.read_csv("test dataset sample.csv")

In [ ]:
df1

,Plaintext,Ciphertext,Cipher Class
0,wearegood,ZICVTZWJH,2
1,meetparty,PHHWSDUWB,0


In [ ]:
df1 = feature_engineering(df1,df1["Ciphertext"])
df1

,Plaintext,Ciphertext,Cipher Class,ic,nuc,A,B,C,D,E,...,Q,R,S,T,U,V,W,X,Y,Z
0,wearegood,ZICVTZWJH,2,0.027778,8,0,0,1,0,0,...,0,0,0,1,0,1,1,0,0,2
1,meetparty,PHHWSDUWB,0,0.055556,7,0,1,0,1,0,...,0,0,1,0,1,0,2,0,0,0


In [ ]:
X_test = df1.loc[:,df1.columns.difference(['Plaintext', 'Ciphertext','Cipher Class'])]
y_test = df1["Cipher Class"]
print(X_test.shape)
print(y_test.shape)
print(X_test.columns)

(2, 28)
(2,)
Index(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
       'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'ic',
       'nuc'],
      dtype='object')


In [ ]:
scaler= StandardScaler()
X_test= scaler.fit_transform(X_test) 
print(X_test)

[[ 0. -1.  1. -1.  0.  0.  0. -1.  1.  1.  0.  0.  0.  0.  0. -1.  0.  0.
  -1.  1. -1.  1. -1.  0.  0.  1. -1.  1.]
 [ 0.  1. -1.  1.  0.  0.  0.  1. -1. -1.  0.  0.  0.  0.  0.  1.  0.  0.
   1. -1.  1. -1.  1.  0.  0. -1.  1. -1.]]


In [ ]:
y_hat = model_BC.predict(X_test)
print(accuracy_score(y_test,y_hat))
print(confusion_matrix(y_test,y_hat))

0.5
[[0 1 0]
 [0 0 0]
 [0 0 1]]
